<img align="center" src="img/course.png" width="800">

# 16720 (B)  Object Tracking in Videos - Assignment 6 - Q3
    Instructor: Kris                          TAs: Wen-Hsuan (Lead), Zen, Yan, Rawal, Paritosh, Qichen

## Q3: Lukas-Kanade on Image Pyramids (Extra Credit, 15 PT)

If the target being tracked moves a lot between frames, the LK tracker can break down. One way to mitigate this problem is to run the LK tracker on a set of image pyramids instead of a single image. The Pyramid tracker starts by performing tracking on a higher level (smaller image) to get a course alignment estimate, propagating this down into the lower level and repeating until a fine aligning warp has been found at the lowest level of the pyramid (the original sized image). In addition to being more robust, the pyramid version of the tracker is much faster because it needs to run fewer gradient descent iterations on the full scale image due to its coarse to fine approach. Implement these modifications for the Lucas-Kanade tracker that you implemented for Q1. You may find this [paper](http://robots.stanford.edu/cs223b04/algo_tracking.pdf) useful.

This question will be graded manually.

<span style='color:red'>**Output:**</span> In your write-up: Please include the relevant code and submit the results of your algorithm on either of the videos, compare with the results from the Lucas-Kanade tracker, and explain any differences.


In [ ]:
def constructPyramid(im, levels):
    pyramid = [im]
    
    for i in range(1, levels):
        h = pyramid[i - 1].shape[1]
        w = pyramid[i - 1].shape[0]
        
        nh = (h + 1) // 2
        nw = (w + 1) // 2
    
        x_o = np.arange(1, w, 2)
        x_e = np.arange(0, w, 2)
        y_o = np.arange(1, h, 2)
        y_e = np.arange(0, h, 2)
        
        if x_e > x_o:
            x_e = x_e[: -1]
        if y_e > y_o:
            y_e = y_e[: -1]
        
        assert x_e == x_o
        assert y_e == y_o
        assert x_e == y_e
        
        im_i = 0.25 * pyramid[i - 1][y_e, x_e] + \
            0.125 * (pyramid[i - 1][y_e, x_o] + pyramid[i - 1][y_e, x_e] + \
                     pyramid[i - 1][y_e, x_e] + pyramid[i - 1][y_e, x_e] )
        

In [ ]:
def LucasKanadePyramidal(it, it1, rect, levels = 3, thresh=.025, maxIters=100):
    '''
    Q1.2: Lucas-Kanade Forward Additive Alignment with Affine MAtrix
    
      Inputs: 
        It: template image
        It1: Current image
        rect: Current position of the object
        (top left, bottom right coordinates, x1, y1, x2, y2)
        levels: Pyramid levels.
        thresh: Stop condition when dp is too small
        maxIters: Maximum number of iterations to run
        
      Outputs:
        M: Affine mtarix (2x3)
    '''

#     M = np.zeros((2, 3))
    M = np.hstack((np.eye(2), np.zeros(2).reshape(-1, 1)))
    x1, y1, x2, y2 = rect
        
    inter_it = RectBivariateSpline(np.arange(it.shape[0]), np.arange(it.shape[1]), it) 
    inter_it1 = RectBivariateSpline(np.arange(it1.shape[0]), np.arange(it1.shape[1]), it1)
    
    x0, y0 = np.meshgrid(np.arange(x1, x2 + 0.5), np.arange(y1, y2 + 0.5))
    x0 = x0.flatten()
    y0 = y0.flatten()  
    
    T = inter_it.ev(y0, x0)
    coords0 = np.hstack((x0.reshape(-1, 1), y0.reshape(-1, 1)))
        
    for i in range(maxIters):

        coords = M @ (np.hstack((coords0, np.ones(coords0.shape[0]).reshape(-1, 1))).T)
        
        x = coords[0].flatten()
        y = coords[1].flatten()
                
        I = inter_it1.ev(y, x)
        
        # Calculating A
        Ix = inter_it1.ev(y, x, dx=0, dy=1)
        Iy = inter_it1.ev(y, x, dx=1, dy=0)
        
        # A = [x.Ix, x.Iy, y.Ix, y.Iy, Ix, Iy]
        A = np.zeros((x.shape[0], 6))
        
        A[:, 0] = x * Ix
        A[:, 1] = x * Iy
        A[:, 2] = y * Ix
        A[:, 3] = y * Iy
        A[:, 4] = Ix
        A[:, 5] = Iy
        
        # Calculating b
        b = T - I
        
        dp = np.linalg.lstsq(A, b, rcond=None)[0]        
        M = M + dp.reshape(np.flip(M.shape)).T
        
        if np.sqrt(np.sum(dp ** 2)) <= thresh:
            break
    
    return M

In [ ]:
# Test your algorithm and visualize results!

# Load data
data_name = 'landing' # could choose from (car1, car2, landing, race, ballet)
data = np.load('./data/%s.npy' % data_name)

# obtain the initial rect with format (x1, y1, x2, y2)
if data_name == 'car1':
    initial = np.array([170, 130, 290, 250])   
elif data_name == 'car2':
    initial = np.array([59, 116, 145, 151])    
elif data_name == 'landing':
    initial = np.array([440, 80, 560, 140])     
elif data_name == 'race':
    initial = np.array([170, 270, 300, 370])
elif data_name == 'ballet':
    initial = np.array([700, 210, 775, 300])     
else:
    assert False, 'the data name must be one of (car1, car2, landing, race, ballet)'

numFrames = data.shape[2]
w = initial[2] - initial[0]
h = initial[3] - initial[1]

# loop over frames
rects = []
rects.append(initial)

for i in range(numFrames-1):

    It = data[:,:,i]
    It1 = data[:,:,i+1]
    rect = rects[i]

    # run algorithm and collect rects
    M = LucasKanadePyramidal(It, It1, rect)
    corners = np.array([[rect[0], rect[1], 1], 
                        [rect[2], rect[3], 1]]).transpose()
    newRect = np.matmul(M, corners).transpose().reshape((4, ))
    rects.append(newRect)

    # Visualize
    fig = plt.figure(1)
    ax = fig.add_subplot(111)
    ax.add_patch(patches.Rectangle((rect[0], rect[1]), rect[2]-rect[0]+1, rect[3]-rect[1]+1, linewidth=2, edgecolor='red', fill=False))
    plt.imshow(It1, cmap='gray')
    plt.show()
    ax.clear()